In [25]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [26]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000670741s, CPU 0.000672s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0147348s, CPU 0.013248s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [27]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneHeatConduction)

Problem("body1-2D", :PlaneHeatConduction, 2, 1, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [28]:
sT = ScalarField(problem, "left", (x, y, z, t) -> 0, steps=100)
sq = ScalarField(problem, "right", (x, y, z, t) -> 100, steps=100)

ScalarField
[[100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0]]

In [29]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=100)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, 100, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [30]:
#T0 = FEM.initialTemperature(problem, "body", T=0)
T0 = scalarField(problem, "body", (x, y, z) -> 0)

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [31]:
#FEM.initialTemperature!("left", T0, T=0)
#T0

In [32]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

ScalarField
[0.0; 6.249999999999992; … ; 0.0; 0.0;;]

In [33]:
#FEM.applyBoundaryConditions!(K, C, q, [supp])

In [34]:
λmax = FEM.largestEigenValue(K, C, support=[supp])

28634.16628528282

In [35]:
ϑ = 0.66
T = FEM.FDM(K, C, q, [supp], T0, 100, 0.0001 * 2 / (abs(1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 0.0 … 0.0 0.0; 0.0 1.9802689762410557e-5 … 0.0019242385683192276 0.0019437046350406712; … ; 0.0 -9.64304029549821e-10 … -9.197460946360748e-8 -9.288127933796567e-8; 0.0 4.6201572046473356e-7 … 4.6535773948001906e-5 4.7022829607872306e-5]

In [36]:
ϑ = 0.33
T2 = FEM.FDM(K, C, q, [supp], T0, 100, 0.0001 * 2 / (abs(1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 0.0 … 0.0 0.0; 0.0 1.863897566632589e-5 … 0.0018120564012237784 0.0018303968062017887; … ; 0.0 -9.077164041532547e-10 … -8.675496428824272e-8 -8.761264033155217e-8; 0.0 4.3474519768725955e-7 … 4.372567032392572e-5 4.418276610661831e-5]

In [37]:
FEM.showDoFResults(T, :T, visible=true)
FEM.showDoFResults(T2)

1

In [38]:
q1 = FEM.solveHeatFlux(T)

VectorField
[[0.0 0.0024703487566296494 … 0.23725730856304966 0.2396282293033439; 0.0 -0.0002900602816860568 … -0.02786115781760835 -0.028139579523783396; … ; 0.0 0.0024703487566296494 … 0.23725730856304966 0.2396282293033439; 0.0 -0.0002900602816860568 … -0.02786115781760835 -0.028139579523783396], [0.0 0.00269334758068857 … 0.25779373494147834 0.26036083950069977; 0.0 -0.0006297131535462438 … -0.0591404985343811 -0.05971773077371609; … ; 0.0 0.00269334758068857 … 0.25779373494147834 0.26036083950069977; 0.0 -0.0006297131535462438 … -0.0591404985343811 -0.05971773077371609], [0.0 0.0023047734799704848 … 0.22143276384809174 0.22364644498691813; 0.0 -0.00023991037997634523 … -0.023631014059626594 -0.02387335617953174; … ; 0.0 0.0023047734799704848 … 0.22143276384809174 0.22364644498691813; 0.0 -0.00023991037997634523 … -0.023631014059626594 -0.02387335617953174], [0.0 5.306735204916842e-7 … 4.8647731121830284e-5 4.910920063058879e-5; 0.0 5.616383549283438e-7 … 4.9503827952616525e-5 4.99

In [39]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [40]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

VectorField
[0.0 2.6654779718013556e-8 … 1.9979073626166758e-6 2.0123018649727303e-6; 0.0 5.462777806139614e-8 … 5.011587875424358e-6 5.059128518839181e-6; … ; 0.0 0.00031037833052085926 … 0.03084214189384692 0.03116110905498373; 0.0 -9.009790923534854e-5 … -0.008416189580899106 -0.008497881471385287]

In [41]:
FEM.showDoFResults(q2, :qvec)

3

In [42]:
q3 = FEM.elementsToNodes(q1)

VectorField
[0.0 2.6654779718013556e-8 … 1.9979073626166758e-6 2.0123018649727303e-6; 0.0 5.462777806139614e-8 … 5.011587875424358e-6 5.059128518839181e-6; … ; 0.0 0.00031037833052085926 … 0.03084214189384692 0.03116110905498373; 0.0 -9.009790923534854e-5 … -0.008416189580899106 -0.008497881471385287]

In [43]:
FEM.showDoFResults(q3, :qvec)

4

In [44]:
err1 = FEM.fieldError(q1)

VectorField
[0.0 5.462777806139625e-8 … 5.0115878754243675e-6 5.05912851883919e-6; 0.0 5.462777806139614e-8 … 5.011587875424358e-6 5.059128518839181e-6; … ; 0.0 0.0014230675444487188 … 0.13652814626805654 0.13789109455027018; 0.0 0.001466582487143563 … 0.13860277548903588 0.1399648123907026]

In [45]:
FEM.showDoFResults(err1, :vector)

5

In [46]:
FEM.openPostProcessor()

XRequest.18: BadValue 0x0


-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [47]:
gmsh.finalize()